In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
def get_vaccine_district_final(date):
    dates_list = [date]
    STATE_NAMES = {
      'AP': 'Andhra Pradesh',
      'AR': 'Arunachal Pradesh',
      'AS': 'Assam',
      'BR': 'Bihar',
      'CT': 'Chhattisgarh',
      'GA': 'Goa',
      'GJ': 'Gujarat',
      'HR': 'Haryana',
      'HP': 'Himachal Pradesh',
      'JH': 'Jharkhand',
      'KA': 'Karnataka',
      'KL': 'Kerala',
      'MP': 'Madhya Pradesh',
      'MH': 'Maharashtra',
      'MN': 'Manipur',
      'ML': 'Meghalaya',
      'MZ': 'Mizoram',
      'NL': 'Nagaland',
      'OR': 'Odisha',
      'PB': 'Punjab',
      'RJ': 'Rajasthan',
      'SK': 'Sikkim',
      'TN': 'Tamil Nadu',
      'TG': 'Telangana',
      'TR': 'Tripura',
      'UT': 'Uttarakhand',
      'UP': 'Uttar Pradesh',
      'WB': 'West Bengal',
      'AN': 'Andaman and Nicobar Islands',
      'CH': 'Chandigarh',
      'DN': 'Dadra and Nagar Haveli and Daman and Diu',
      'DL': 'Delhi',
      'JK': 'Jammu and Kashmir',
      'LA': 'Ladakh',
      'LD': 'Lakshadweep',
      'PY': 'Puducherry',
      'TT': 'India',
     # [UNASSIGNED_STATE_CODE]: 'Unassigned',
    }

    #Change the date here to get historical dates
    # date_=datetime.strptime('2021-12-04',"%Y-%m-%d")
    # dates_list= [date_+timedelta(days=i) for i in range(4)]
    # dates_list

    source_path="../RAWCSV"

    for Date in dates_list:
        tdate = Date
        date=tdate.strftime("%d/%m/%Y")
        if not isinstance(Date,str):
            Date=Date.strftime("%Y-%m-%d")
        print(Date)
        # /home/swiadmin/test/csv/latest/
        cowin_vaccine_data_districtwise_prev=pd.read_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",header=[0,1])
        try:
            cowin_vaccine_data_districtwise_prev = cowin_vaccine_data_districtwise_prev.drop([date],axis=1)
        except:
            pass
        cowin_vaccine_data_districtwise_master=pd.DataFrame()
        for state in STATE_NAMES.keys():
            if state=="TT":
                continue
    #         print(state)
            state_=pd.read_csv(f"{source_path}/{Date}/{state}_final.csv")
            state_.dropna(subset=["District"],axis=0,inplace=True)
            state_.reset_index(inplace=True,drop=True)
            temp_state_df=pd.DataFrame()
            temp_state_df["District"]=state_["District"]
            temp_state_df["State_Code"]=state
            temp_state_df["State"]=temp_state_df["State_Code"].map(STATE_NAMES)
            temp_state_df["District_Key"]=temp_state_df.apply(lambda rw: rw["State_Code"]+"_"+rw["District"],axis=1)
            temp_state_df["First Dose Administered"]=state_["cumulativeVaccinated1NumberForDistrict"]
            temp_state_df["Second Dose Administered"]=state_["cumulativeVaccinated2NumberForDistrict"]
            temp_state_df["Total Dose Administered"]=temp_state_df["First Dose Administered"]+temp_state_df["Second Dose Administered"]
            cowin_vaccine_data_districtwise_master=pd.concat([cowin_vaccine_data_districtwise_master,temp_state_df],axis=0)

        # if not isinstance(Date,str):


        columns=pd.MultiIndex.from_tuples(zip(["District","State_Code","State",
                    "District_Key",date,date,date],
                   [" "," "," "," ","Total Doses Administered",
                    "First Dose Administered","Second Dose Administered"
                   ]))
        cowin_vaccine_data_districtwise_master.columns=columns
        cowin_vaccine_data_districtwise_master.reset_index(inplace=True,drop=True)

        cowin_vaccine_data_districtwise_latest=pd.merge(cowin_vaccine_data_districtwise_prev,cowin_vaccine_data_districtwise_master,
                                                       left_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                       right_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                       how="left")
        cowin_vaccine_data_districtwise_latest = cowin_vaccine_data_districtwise_latest.drop_duplicates()
        cowin_vaccine_data_districtwise_latest.to_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",index=False)



In [1]:
# Version 2.0

In [4]:
from datetime import timedelta
from ratelimit import limits, sleep_and_retry
import requests

@sleep_and_retry
@limits(calls=1, period=timedelta(seconds=60).total_seconds())
def get_cowin_district_limit(url):
    response = requests.get(url)
    # response.raise_for_status()
    return response

In [5]:
get_cowin_district_limit()

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.26.0',
  'X-Amzn-Trace-Id': 'Root=1-621db053-63278fe20e55e71d4ea2fed3'},
 'origin': '101.53.158.87',
 'url': 'https://httpbin.org/get'}

In [ ]:
def vaccination_numbers_for_districts(state_id, district_id, Date):
    
    if not isinstance(Date,str):
        Date=Date.strftime("%Y-%m-%d")
    
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
    # print(cowin_codes)

    state_cumulative_vaccinated1=0
    First_Dose_Administered=0
    Second_Dose_Administered=0
    Total_Doses_Administered=0
    Sessions=0
    Sites=0
    Male_Doses_Administered=0
    Female_Doses_Administered=0
    Transgender_Doses_Administered=0
    Covaxin_Doses_Administered=0
    CoviShield_Doses_Administered=0

    api_url_district="https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?state_id={s_id}&district_id={d_id}&date={date}".format(s_id = str(state_id), d_id = str(district_id),date = Date)
    print(api_url_district)
    api_data_district = requests.get(api_url_district)
    print(api_data_district)
    First_Dose_Administered=api_data_district.json()["topBlock"]["vaccination"]["tot_dose_1"] 
    Second_Dose_Administered=api_data_district.json()["topBlock"]["vaccination"]["tot_dose_2"]

    Total_Doses_Administered=api_data_district.json()["topBlock"]["vaccination"]["total"] 
    Sessions=api_data_district.json()["topBlock"]["sessions"]["total"] 
    Sites=api_data_district.json()["topBlock"]["sites"]["total"] 

    Male_Doses_Administered = api_data_district.json()["topBlock"]["vaccination"]["male"] 
    Female_Doses_Administered=api_data_district.json()["topBlock"]["vaccination"]["female"]
    Transgender_Doses_Administered=api_data_district.json()["topBlock"]["vaccination"]["others"] 
    Covaxin_Doses_Administered=api_data_district.json()["topBlock"]["vaccination"]["covaxin"] 
    CoviShield_Doses_Administered=api_data_district.json()["topBlock"]["vaccination"]["covishield"] 

    return (First_Dose_Administered,Second_Dose_Administered,Total_Doses_Administered,
            Sessions,Sites,Male_Doses_Administered,Female_Doses_Administered,Transgender_Doses_Administered,
           Covaxin_Doses_Administered,CoviShield_Doses_Administered)

def get_vaccine_district_csv(date):
    dates_list = [date]
    
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
        
    df_prev=pd.read_csv("/mnt/c/Users/91967/Documents/harika/harika/swiworkspace/incovid19_vaccine_districtwise/cowin_vaccine_data_districtwise.csv")
    for Date in dates_list:
        i=0
        df = pd.DataFrame()
        for index,row in cowin_codes.iterrows():
            
            state_id = row['state_id']
            district_id = row['district_id']
            state_name = row['state_name']
            district_name = row['district_name']
            print(state_id,district_id,state_name,district_name)
        
            columns=pd.MultiIndex.from_tuples(zip(["District","State",date,date,date,date,date,date,date,date,date,date],
            [" "," ",
            "Total Individuals Registered","Sessions","Sites","First Dose Administered",
            "Second Dose Administered","Male(Doses Administered)","Female(Doses Administered)","Transgender(Doses Administered)",
            "Covaxin(Doses Administered)","CoviShield(Doses Administered)"]))
            

            vacc_data = vaccination_numbers_for_districts(state_id,district_id,Date)
            print('vacc_data',vacc_data, state_id, district_id)
            df.loc[i,"State"]= state_name
            df.loc[i,"District"] = district_name
            
            df.loc[i,"Total Individuals Registered"]=vacc_data[2]
            df.loc[i,"Sessions"]=vacc_data[3]
            df.loc[i,"Sites"]=vacc_data[4]
            df.loc[i,"First Dose Administered"]=vacc_data[0]
            df.loc[i,"Second Dose Administered"]=vacc_data[1]
            df.loc[i,"Male(Doses Administered)"]=vacc_data[5]
            df.loc[i,"Female(Doses Administered)"]=vacc_data[6]
            df.loc[i,"Transgender(Doses Administered)"]=vacc_data[7]
            Covaxin = df.loc[i,"Covaxin(Doses Administered)"]=vacc_data[8]
            CoviShield = df.loc[i,"CoviShield(Doses Administered)"]=vacc_data[9]
            i+=1

            # break
    df.columns = columns
    df.reset_index(inplace=True,drop=True)
    
    df.to_csv("/mnt/c/Users/91967/Documents/harika/harika/swiworkspace/incovid19_vaccine_districtwise/cowin_vaccine_data_districtwise4.csv",mode='a')